In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import Callback
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf

In [9]:
file_path = '../../Data/Verified/BaseLineLabeledWithoutSelfLabeledValues/'
file_name = 'RecordedDrivingData.csv'

recorded_driving_dataframe = pd.read_csv(file_path + file_name)

recorded_driving_dataframe['Timestamp'] = pd.to_datetime(recorded_driving_dataframe['Timestamp'])

primary_data = recorded_driving_dataframe[['Lateral acceleration','Longitudinal acceleration']]
full_data = recorded_driving_dataframe
data_labels = recorded_driving_dataframe['Label']
data_labels = data_labels.loc[primary_data.index]

In [11]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data_labels)

Train_Data, Test_Data, Train_Labels, Test_Labels = train_test_split(primary_data, encoded_labels, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, activation='relu', input_shape=(Train_Data.shape[1],)),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

color_map_for_plot = {
    'Excellent': 'green',
    'Acceptable': 'yellow',
    'So and So': 'orange',
    'Uncomfortable': 'red',
}

class ActivationLogger(Callback):
    def __init__(self, layer_index):
        super(ActivationLogger, self).__init__()
        self.layer_index = layer_index

    def on_epoch_end(self, epoch, logs=None):
        layer_output_model = tf.keras.Model(inputs=self.model.input,
                                            outputs=self.model.layers[self.layer_index].output)
        activations = layer_output_model.predict(self.model.input)
        zero_activations = np.sum(activations == 0)
        total_activations = activations.size
        zero_fraction = zero_activations / total_activations
        print(f'Epoch {epoch+1}: Layer {self.layer_index} - Zero activations: {zero_fraction * 100:.2f}%')


class PredictionCallback(Callback):
    def __init__(self, Test_Data, Test_Labels, label_encoder, dataframe, color_map):
        self.Test_Data = Test_Data
        self.Test_Labels = Test_Labels
        self.label_encoder = label_encoder
        self.dataframe = dataframe
        self.color_map = color_map

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 50 == 0:
            test_data_predictions = self.model.predict(self.Test_Data)
            test_labels_predictions = np.argmax(test_data_predictions, axis=1)
            true_labels = self.Test_Labels

            accuracy_per_epoch = np.mean(test_labels_predictions == true_labels)
            loss_per_epoch = logs['val_loss']
            print(f'Epoch: {epoch + 1}')
            print(f'Accuracy: {accuracy_per_epoch}')
            print(f'Loss: {loss_per_epoch}')

            test_labels_predictions_mapped = self.label_encoder.inverse_transform(test_labels_predictions)

            fig, axis = plt.subplots()
            fig.set_size_inches(4, 4)

            plot_dataframe = pd.DataFrame({
                'Lateral acceleration': self.Test_Data[:, 0],
                'Longitudinal acceleration': self.Test_Data[:, 1],
                'Label': test_labels_predictions_mapped
            })

            colors = plot_dataframe['Label'].map(self.color_map)

            axis.scatter(plot_dataframe['Lateral acceleration'], plot_dataframe['Longitudinal acceleration'], c=colors)

            max_lateral = plot_dataframe['Lateral acceleration'].max()
            max_longitudinal = plot_dataframe['Longitudinal acceleration'].max()
            maximum_axis_value = max(max_lateral, max_longitudinal)

            axis.set_xlim(-maximum_axis_value - 0.5, maximum_axis_value + 0.5)
            axis.set_ylim(-maximum_axis_value - 0.5, maximum_axis_value + 0.5)

            plt.grid(True)
            plt.xlabel('Lateral acceleration')
            plt.ylabel('Longitudinal acceleration')
            plt.title(f'Predicted Labels By Model After {epoch + 1} Epochs')
            plt.show()

# Combine both callbacks
callbacks = [
    ActivationLogger(layer_index=1),
    PredictionCallback(Test_Data, Test_Labels, label_encoder, primary_data, color_map_for_plot)
]

# Train the model with both callbacks
model_history = model.fit(Train_Data, Train_Labels, epochs=20, batch_size=32, validation_split=0.2, callbacks=callbacks)

# Evaluate the model
loss, accuracy = model.evaluate(Test_Data, Test_Labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [34]:
print(Test_Data)

In [31]:
# Make predictions on the test data
test_data_predictions = model.predict(Test_Data)
test_labels_predictions = np.argmax(test_data_predictions, axis=1)

# Evaluate the model
loss, accuracy = model.evaluate(Test_Data, Test_Labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Create confusion matrix
cm = confusion_matrix(Test_Labels, test_labels_predictions)
cm_df = pd.DataFrame(cm, index=label_encoder.classes_, columns=label_encoder.classes_)

plt.figure(figsize=(10, 7))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Assuming Test_Labels and test_labels_predictions are already defined

# Get precision, recall, and f1-score
precision, recall, f1, _ = precision_recall_fscore_support(Test_Labels, test_labels_predictions, average=None)

specificity = []
for i in range(len(cm)):
    tn = np.sum(cm) - (np.sum(cm[:, i]) + np.sum(cm[i, :]) - cm[i, i])
    fp = np.sum(cm[:, i]) - cm[i, i]
    specificity.append(tn / (tn + fp))

# Create a DataFrame for the scores
labels = label_encoder.inverse_transform([0, 1, 2, 3])
scores_df = pd.DataFrame({
    'Label': labels,
    'Precision (%)': precision * 100,
    'Recall (%)': recall * 100,
    'F1-Score (%)': f1 * 100,
    'Specificity (%)': np.array(specificity) * 100
})

# Create a DataFrame for the scores
labels = label_encoder.inverse_transform([0, 1, 2, 3])
scores_df = pd.DataFrame({
    'Label': labels,
    'Precision (%)': (precision * 100).round(2),
    'Recall (%)': (recall * 100).round(2),
    'F1-Score (%)': (f1 * 100).round(2),
    'Specificity (%)': (np.array(specificity) * 100).round(2)
})

# Plotting the table
fig, ax = plt.subplots()
ax.axis('tight')
ax.axis('off')
ax.table(cellText=scores_df.values, colLabels=scores_df.columns, cellLoc='center', loc='center', bbox=[0, 0, 1, 1])

plt.title('Classification Report', pad=20)
plt.show()

In [32]:
# model_json_config = model.to_json()
# 
# directory = '../../AIModels/BaselineModel/Epochs/'
# os.makedirs(directory, exist_ok=True)
# 
# with open(directory + 'DrivingComfortabilityPredictingModel.json', 'w') as json_file:
#     json_file.write(model_json_config)
#     
# model.save_weights(directory +  'DrivingComfortabilityPredictingModel.weights.h5')
# 
# with open(directory + 'DrivingComfortabilityPredictingModelHistory.pkl', 'wb') as f:
#     pickle.dump(model_history.history, f)

In [33]:
model.save('../../AIModels/BaselineModel/20Epochs/FullModel/DrivingComfortabilityPredictingModel.keras')